In [1]:
import os.path as osp
import argparse

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch_geometric.datasets import Planetoid, Coauthor, Amazon
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GAE, VGAE, APPNP
import torch_geometric.transforms as T

from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import pandas as pd
import numpy as np
from torch_geometric.data import Data

In [2]:
parser = argparse.ArgumentParser()
parser.add_argument('--model', type=str, default='VGNAE')
parser.add_argument('--dataset', type=str, default='wiki')
parser.add_argument('--epochs', type=int, default=500)
parser.add_argument('--channels', type=int, default=128)
parser.add_argument('--scaling_factor', type=float, default=1.8)
parser.add_argument('--training_rate', type=float, default=0.9) 
args,_ = parser.parse_known_args()

In [3]:
nodes_number = 2405

filename_adj = "datasets/graph.txt"

raw_edges = pd.read_csv(filename_adj, header=None, sep='\t')

drop_self_loop = raw_edges[raw_edges[0]!=raw_edges[1]]

graph_np=np.zeros((nodes_number, nodes_number))

for i in range(drop_self_loop.shape[0]):
    graph_np[drop_self_loop.iloc[i,0],drop_self_loop.iloc[i,1]]=1
    graph_np[drop_self_loop.iloc[i,1],drop_self_loop.iloc[i,0]]=1
    
edges = torch.tensor([list(graph_np.nonzero()[0]),list(graph_np.nonzero()[1])])

features = torch.eye(nodes_number)

data = Data(x=features, edge_index=edges)

data = T.NormalizeFeatures()(data)

In [4]:
class Encoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels, edge_index):
        super(Encoder, self).__init__()
        self.linear1 = nn.Linear(in_channels, out_channels)
        self.linear2 = nn.Linear(in_channels, out_channels)
        self.propagate = APPNP(K=1, alpha=0)

    def forward(self, x, edge_index,not_prop=0):
        if args.model == 'GNAE':
            x = self.linear1(x)
            x = F.normalize(x,p=2,dim=1)  * args.scaling_factor
            x = self.propagate(x, edge_index)
            return x

        if args.model == 'VGNAE':
            x_ = self.linear1(x)
            x_ = self.propagate(x_, edge_index)

            x = self.linear2(x)
            x = F.normalize(x,p=2,dim=1) * args.scaling_factor
            x = self.propagate(x, edge_index)
            return x, x_

        return x

In [5]:
dev = torch.device('cpu')
channels = args.channels
train_rate = args.training_rate
val_ratio = (1-args.training_rate) / 3
test_ratio = (1-args.training_rate) / 3 * 2
data = train_test_split_edges(data.to(dev), val_ratio=val_ratio, test_ratio=test_ratio)

N = int(data.x.size()[0])
if args.model == 'GNAE':   
    model = GAE(Encoder(data.x.size()[1], channels, data.train_pos_edge_index)).to(dev)
if args.model == 'VGNAE':
    model = VGAE(Encoder(data.x.size()[1], channels, data.train_pos_edge_index)).to(dev)

data.train_mask = data.val_mask = data.test_mask = data.y = None
x, train_pos_edge_index = data.x.to(dev), data.train_pos_edge_index.to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

C:\Users\zhangshanfan\.conda\envs\pytorch\lib\site-packages\torch_geometric\deprecation.py:13: UserWarning: 'train_test_split_edges' is deprecated, use 'transforms.RandomLinkSplit' instead
  warnings.warn(out)


In [6]:
def computer_indi(z, pos_edge_index, neg_edge_index, model):
    pos_y = z.new_ones(pos_edge_index.size(1))
    neg_y = z.new_zeros(neg_edge_index.size(1))
    y = torch.cat([pos_y, neg_y], dim=0)

    pos_pred = model.decoder(z, pos_edge_index, sigmoid=True)
    neg_pred = model.decoder(z, neg_edge_index, sigmoid=True)
    pred = torch.cat([pos_pred, neg_pred], dim=0)

    y, pred = y.detach().cpu().numpy(), pred.detach().cpu().numpy()
    
    auc = roc_auc_score(y, pred)
    ap = average_precision_score(y, pred)

    return ap, auc

In [7]:
def train():
    model.train()
    optimizer.zero_grad()
    z  = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    if args.model in ['VGAE']:
        loss = loss + (1 / data.num_nodes) * model.kl_loss()
    loss.backward()
    optimizer.step()
    return loss

def test(pos_edge_index, neg_edge_index, plot_his=0):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return computer_indi(z, pos_edge_index, neg_edge_index, model)

In [8]:
for epoch in range(1,args.epochs):
    loss = train()
    loss = float(loss)
    
    with torch.no_grad():
        test_pos, test_neg = data.test_pos_edge_index, data.test_neg_edge_index
        ap, auc = test(data.test_pos_edge_index, data.test_neg_edge_index)
        print('Epoch: {:03d}, AP: {:.4f}, AUC: {:.4f}'.format(epoch, ap, auc))

Epoch: 001, AP: 0.8238, AUC: 0.8215
Epoch: 002, AP: 0.8325, AUC: 0.8308
Epoch: 003, AP: 0.8387, AUC: 0.8378
Epoch: 004, AP: 0.8439, AUC: 0.8435
Epoch: 005, AP: 0.8488, AUC: 0.8488
Epoch: 006, AP: 0.8537, AUC: 0.8542
Epoch: 007, AP: 0.8586, AUC: 0.8593
Epoch: 008, AP: 0.8638, AUC: 0.8648
Epoch: 009, AP: 0.8693, AUC: 0.8706
Epoch: 010, AP: 0.8750, AUC: 0.8764
Epoch: 011, AP: 0.8813, AUC: 0.8825
Epoch: 012, AP: 0.8875, AUC: 0.8885
Epoch: 013, AP: 0.8941, AUC: 0.8946
Epoch: 014, AP: 0.9012, AUC: 0.9006
Epoch: 015, AP: 0.9077, AUC: 0.9058
Epoch: 016, AP: 0.9140, AUC: 0.9101
Epoch: 017, AP: 0.9188, AUC: 0.9129
Epoch: 018, AP: 0.9231, AUC: 0.9142
Epoch: 019, AP: 0.9269, AUC: 0.9144
Epoch: 020, AP: 0.9305, AUC: 0.9156
Epoch: 021, AP: 0.9334, AUC: 0.9171
Epoch: 022, AP: 0.9361, AUC: 0.9188
Epoch: 023, AP: 0.9379, AUC: 0.9201
Epoch: 024, AP: 0.9395, AUC: 0.9213
Epoch: 025, AP: 0.9405, AUC: 0.9220
Epoch: 026, AP: 0.9412, AUC: 0.9225
Epoch: 027, AP: 0.9418, AUC: 0.9225
Epoch: 028, AP: 0.9423, AUC:

Epoch: 229, AP: 0.9450, AUC: 0.9278
Epoch: 230, AP: 0.9448, AUC: 0.9276
Epoch: 231, AP: 0.9448, AUC: 0.9276
Epoch: 232, AP: 0.9449, AUC: 0.9276
Epoch: 233, AP: 0.9448, AUC: 0.9274
Epoch: 234, AP: 0.9447, AUC: 0.9273
Epoch: 235, AP: 0.9447, AUC: 0.9273
Epoch: 236, AP: 0.9447, AUC: 0.9274
Epoch: 237, AP: 0.9448, AUC: 0.9275
Epoch: 238, AP: 0.9448, AUC: 0.9275
Epoch: 239, AP: 0.9448, AUC: 0.9274
Epoch: 240, AP: 0.9448, AUC: 0.9274
Epoch: 241, AP: 0.9448, AUC: 0.9274
Epoch: 242, AP: 0.9448, AUC: 0.9273
Epoch: 243, AP: 0.9448, AUC: 0.9273
Epoch: 244, AP: 0.9447, AUC: 0.9272
Epoch: 245, AP: 0.9447, AUC: 0.9272
Epoch: 246, AP: 0.9447, AUC: 0.9273
Epoch: 247, AP: 0.9447, AUC: 0.9273
Epoch: 248, AP: 0.9448, AUC: 0.9274
Epoch: 249, AP: 0.9448, AUC: 0.9275
Epoch: 250, AP: 0.9449, AUC: 0.9276
Epoch: 251, AP: 0.9450, AUC: 0.9278
Epoch: 252, AP: 0.9451, AUC: 0.9278
Epoch: 253, AP: 0.9452, AUC: 0.9279
Epoch: 254, AP: 0.9453, AUC: 0.9281
Epoch: 255, AP: 0.9454, AUC: 0.9282
Epoch: 256, AP: 0.9453, AUC:

Epoch: 457, AP: 0.9465, AUC: 0.9267
Epoch: 458, AP: 0.9464, AUC: 0.9265
Epoch: 459, AP: 0.9463, AUC: 0.9264
Epoch: 460, AP: 0.9462, AUC: 0.9263
Epoch: 461, AP: 0.9461, AUC: 0.9261
Epoch: 462, AP: 0.9459, AUC: 0.9258
Epoch: 463, AP: 0.9458, AUC: 0.9256
Epoch: 464, AP: 0.9457, AUC: 0.9254
Epoch: 465, AP: 0.9456, AUC: 0.9253
Epoch: 466, AP: 0.9455, AUC: 0.9251
Epoch: 467, AP: 0.9454, AUC: 0.9250
Epoch: 468, AP: 0.9454, AUC: 0.9250
Epoch: 469, AP: 0.9455, AUC: 0.9251
Epoch: 470, AP: 0.9455, AUC: 0.9251
Epoch: 471, AP: 0.9456, AUC: 0.9252
Epoch: 472, AP: 0.9457, AUC: 0.9253
Epoch: 473, AP: 0.9458, AUC: 0.9255
Epoch: 474, AP: 0.9460, AUC: 0.9257
Epoch: 475, AP: 0.9462, AUC: 0.9260
Epoch: 476, AP: 0.9463, AUC: 0.9261
Epoch: 477, AP: 0.9464, AUC: 0.9263
Epoch: 478, AP: 0.9465, AUC: 0.9263
Epoch: 479, AP: 0.9465, AUC: 0.9264
Epoch: 480, AP: 0.9465, AUC: 0.9264
Epoch: 481, AP: 0.9465, AUC: 0.9263
Epoch: 482, AP: 0.9465, AUC: 0.9263
Epoch: 483, AP: 0.9463, AUC: 0.9261
Epoch: 484, AP: 0.9462, AUC: